### Chain Using LangGraph
In this section we will see how we can build a simple chain using Langgraph that uses 4 important concepts

- How to use chat messages as our graph state
- How to use chat models in graph nodes
- How to bind tools to our LLM in chat models
- How to execute the tools call in our graph nodes 

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()


os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

#### How to use chat messages as our graph state
##### Messages

We can use messages which can be used to capture different roles within a conversation.
LangChain has various message types including HumanMessage, AIMessage, SystemMessage and ToolMessage.
These represent a message from the user, from chat model, for the chat model to instruct behavior, and from a tool call.

Every message have these important components.

- content - content of the message
- name - Specify the name of author
- response_metadata - optionally, a dict of metadata (e.g., often populated by model provider for AIMessages)



In [8]:
from langchain_core.messages import AIMessage, HumanMessage
from pprint import pprint

messages = [AIMessage(content = f"Please tell me how can I help you!", name="LLMModel")]
messages.append(HumanMessage(content=f"I want to learn coding", name="Tejas"))
messages.append(AIMessage(content=f"Which programming language you want to learn", name="LLMModel"))
messages.append(HumanMessage(content=f"I want to learn Python Programming Language", name="Tejas"))

for message in messages:
    message.pretty_print()

================================== Ai Message ==================================
Name: LLMModel

Please tell me how can I help you!
================================ Human Message =================================
Name: Tejas

I want to learn coding
================================== Ai Message ==================================
Name: LLMModel

Which programming language you want to learn
================================ Human Message =================================
Name: Tejas

I want to learn Python Programming Language


### Chat Models

We can use the sequence of message as input with the chatmodels using LLM's and OPENAI.

In [10]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4o-mini", temperature=0)
response = llm.invoke(messages)
print(response.content)

Great choice! Python is a versatile and beginner-friendly programming language. Here’s a step-by-step guide to help you get started:

### 1. **Set Up Your Environment**
   - **Install Python**: Download and install Python from the [official website](https://www.python.org/downloads/). Make sure to check the box that says "Add Python to PATH" during installation.
   - **Choose an IDE/Text Editor**: You can use any text editor, but popular choices include:
     - **VS Code**: A powerful code editor with many extensions.
     - **PyCharm**: A dedicated Python IDE.
     - **Jupyter Notebook**: Great for data science and interactive coding.

### 2. **Learn the Basics**
   - **Syntax and Data Types**: Understand variables, strings, integers, floats, lists, tuples, and dictionaries.
   - **Control Structures**: Learn about `if` statements, loops (`for`, `while`), and how to control the flow of your program.
   - **Functions**: Learn how to define and call functions, and understand the concept

In [11]:
response.response_metadata

{'token_usage': {'completion_tokens': 618,
  'prompt_tokens': 61,
  'total_tokens': 679,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4o-mini-2024-07-18',
 'system_fingerprint': 'fp_8bda4d3a2c',
 'id': 'chatcmpl-CD4Abvtfa2uu8RInvCnrgyz44wUBN',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}

### Tools
Tools can be integrated with the LLM models to interact with external systems. External systems can be API's, third party tools.

Whenever a query is asked the model can choose to call the tool and this query is based on the 
natural language input and this will return an output that matches the tool's schema

In [19]:
def add(a:int,b:int)-> int:
    """ Add a and b
    Args:
        a (int): first int
        b (int): second int

    Returns:
        int
    """
    return a+b

In [13]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000019AA4E7F4D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000019AA492C2C0>, root_client=<openai.OpenAI object at 0x0000019AA3535E80>, root_async_client=<openai.AsyncOpenAI object at 0x0000019AA4E7C590>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [20]:
### Binding tool with llm

llm_with_tools=llm.bind_tools([add])

tool_call=llm_with_tools.invoke([HumanMessage(content=f"What is 2 plus 2",name="Krish")])

In [21]:
tool_call.tool_calls

[{'name': 'add',
  'args': {'a': 2, 'b': 2},
  'id': 'call_1JSXS6xslcmp68k1jShra7BR',
  'type': 'tool_call'}]